### *Localisation optimale des capacités de production éoliennes en Europe*

Notebook pour le  chargement des données du projet (partie I) -- version 1.2 (màj le 31/3/2020)

In [2]:
## imports classiques
import numpy as np

from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPArray, CyLPModel

import matplotlib.pyplot as plt
import math 
import pandas as pd
import plotly.graph_objects as go
import matplotlib.colors as colors
import random


colors_list = list(colors._colors_full_map.values())
random.shuffle(colors_list)

%matplotlib notebook

In [158]:
## paramètres

liste_pays = ['Allemagne','Autriche','Belgique','Danemark','Espagne','France','Irlande','Italie','Luxembourg',\
             'Norvège','Pays-Bas','Portugal','Royaume-Uni','Suède','Suisse']

liste_pays_index_dic = {'Allemagne':0,'Autriche':1,'Belgique':2,'Danemark':3,'Espagne':4,'France':5,'Irlande':6,'Italie':7,'Luxembourg':8,\
             'Norvège':9,'Pays-Bas':10,'Portugal':11,'Royaume-Uni':12,'Suède':13,'Suisse':14}

list_colors_dic = {'Allemagne':colors_list[0],'Autriche':colors_list[1],'Belgique':colors_list[2],'Danemark':colors_list[3],'Espagne':colors_list[4],\
                         'France':colors_list[5],'Irlande':colors_list[6],'Italie':colors_list[7],'Luxembourg':colors_list[8],\
             'Norvège':colors_list[9],'Pays-Bas':colors_list[10],'Portugal':colors_list[11],'Royaume-Uni':colors_list[12],'Suède':colors_list[13],'Suisse':colors_list[14]} 

TURB = np.array([8587 , 12009 , 1417 , 9 , 18372 , 25132 , 527 , 21117 , 1140 , 28941 , 37 , 5052 , 4269 , 16637 , 15101])
POMP = np.array([5223 , 3580 , 1307 , 0 , 5347 , 4303 , 292 , 7544 , 1100 , 1396 , 0 , 1029 , 2744 , 45 , 1636 ])
RES = np.array([0.3*1e6 , 3.2*1e6 , 0.01*1e6 , 0 , 18.4*1e6 , 9.8*1e6 , 0.24*1e6 , 7.9*1e6 , 0.005*1e6 , 84.147*1e6 , 0 , 2.6*1e6 , 1.2*1e6 , 33.756*1e6 , 8.4*1e6])

N_pays = 15                  # plus petit ou égal à N_pays_max = 15
N_sites = 642                # plus petit ou égal à sites_max = 642
N_heures_par_annee = 24*365  # nombre d'heures par année
N_annees = 1/30              # fraction de l'annee à utiliser ; prendre par exemple 1/12 pour tester sur un mois (calculs 
                             # plus rapides) ; dans le rapport il faut utiliser l'année complète N_annees = 1
N_heures = int(np.ceil(N_heures_par_annee*N_annees)) # nombre d'heures à extraire des fichiers de rendements

vecteur_temps = np.arange(1,N_heures+1) / N_heures_par_annee

In [159]:
## chargement et traitement des données brutes (I)
 
sites = pd.read_csv('Sites.csv')

onshore = sites[sites['capacite offshore']=='Non'].copy()
offshore = sites[sites['capacite offshore']=='Oui'].copy()

In [160]:
sites.head()

,index site,latitude,longitude,pays,couleur,capacite offshore,scores,capacites
0,1,54.0,8.0,Allemagne,#7b0323,Non,0.003291,7509.607003
1,2,54.0,9.0,Allemagne,#7b0323,Non,0.002193,5003.709632
2,3,54.0,10.0,Allemagne,#7b0323,Non,0.001852,4225.781588
3,4,54.0,11.0,Allemagne,#7b0323,Non,0.001852,4225.781588
4,5,54.0,12.0,Allemagne,#7b0323,Non,0.002193,5003.709632


In [161]:
onshore.head()

,index site,latitude,longitude,pays,couleur,capacite offshore,scores,capacites
0,1,54.0,8.0,Allemagne,#7b0323,Non,0.003291,7509.607003
1,2,54.0,9.0,Allemagne,#7b0323,Non,0.002193,5003.709632
2,3,54.0,10.0,Allemagne,#7b0323,Non,0.001852,4225.781588
3,4,54.0,11.0,Allemagne,#7b0323,Non,0.001852,4225.781588
4,5,54.0,12.0,Allemagne,#7b0323,Non,0.002193,5003.709632


In [162]:
offshore.head()

,index site,latitude,longitude,pays,couleur,capacite offshore,scores,capacites
487,0,54.0,7.0,Allemagne,#7b0323,Oui,0.006342,8364.150000
488,48,56.0,5.0,Danemark,#A9A9A9,Oui,0.010571,15478.589744
489,49,55.0,6.0,Allemagne,#7b0323,Oui,0.010571,13940.250000
490,61,52.0,3.0,Pays-Bas,#04d8b2,Oui,0.006342,2349.264706
491,66,58.0,10.0,Suède,#758da3,Oui,0.006342,8577.125000


In [163]:
"""# Visualisations des données pour les sites 

fig = go.Figure(data=go.Scattergeo(
        lon = offshore['longitude'],
        lat = offshore['latitude'],
        text = offshore['pays'],
        mode = 'markers',
        marker=dict(
        color = offshore['couleur'],
        size = offshore['scores']*1000)
        ))

fig.update_layout(
        title = 'Sites offshore, et leur importance',
        geo_scope = 'europe'
    )
fig.show();"""

"# Visualisations des données pour les sites \n\nfig = go.Figure(data=go.Scattergeo(\n        lon = offshore['longitude'],\n        lat = offshore['latitude'],\n        text = offshore['pays'],\n        mode = 'markers',\n        marker=dict(\n        color = offshore['couleur'],\n        size = offshore['scores']*1000)\n        ))\n\nfig.update_layout(\n        title = 'Sites offshore, et leur importance',\n        geo_scope = 'europe'\n    )\nfig.show();"

In [164]:
"""fig = go.Figure(data=go.Scattergeo(
        lon = onshore['longitude'],
        lat = onshore['latitude'],
        text = onshore['pays'],
        mode = 'markers',
        marker=dict(
        color = onshore['couleur'],
        size = onshore['scores']*3000)
        ))

fig.update_layout(
        title = 'Sites onshore, et leur importance',
        geo_scope = 'europe'
    )
fig.show();"""

"fig = go.Figure(data=go.Scattergeo(\n        lon = onshore['longitude'],\n        lat = onshore['latitude'],\n        text = onshore['pays'],\n        mode = 'markers',\n        marker=dict(\n        color = onshore['couleur'],\n        size = onshore['scores']*3000)\n        ))\n\nfig.update_layout(\n        title = 'Sites onshore, et leur importance',\n        geo_scope = 'europe'\n    )\nfig.show();"

In [165]:
"""fig = go.Figure(data=go.Scattergeo(
        lon = sites['longitude'],
        lat = sites['latitude'],
        text = sites['pays'],
        mode = 'markers',
        marker_color = sites['couleur'],
        marker = dict(size = 2),
        ))

fig.update_layout(
        title = 'Affectation par pays des sites éoliens',
        geo_scope = 'europe'
    )
fig.show();"""

"fig = go.Figure(data=go.Scattergeo(\n        lon = sites['longitude'],\n        lat = sites['latitude'],\n        text = sites['pays'],\n        mode = 'markers',\n        marker_color = sites['couleur'],\n        marker = dict(size = 2),\n        ))\n\nfig.update_layout(\n        title = 'Affectation par pays des sites éoliens',\n        geo_scope = 'europe'\n    )\nfig.show();"

In [166]:
"""fig = go.Figure(data=go.Scattergeo(
        lon = sites['longitude'],
        lat = sites['latitude'],
        text = 'pays: '+sites['pays']+', offshore: '+sites['capacite offshore']+' | capacite [MW] = '+\
        sites['capacites'].astype(str),
        mode = 'markers',
        marker = dict(
            color = sites['couleur'],
            size = sites['capacites']/sites['capacites'].max() * 30)
        ))

fig.update_layout(
        title = 'Capacites des sites éoliens',
        geo_scope = 'europe'
    )
fig.show();"""

"fig = go.Figure(data=go.Scattergeo(\n        lon = sites['longitude'],\n        lat = sites['latitude'],\n        text = 'pays: '+sites['pays']+', offshore: '+sites['capacite offshore']+' | capacite [MW] = '+        sites['capacites'].astype(str),\n        mode = 'markers',\n        marker = dict(\n            color = sites['couleur'],\n            size = sites['capacites']/sites['capacites'].max() * 30)\n        ))\n\nfig.update_layout(\n        title = 'Capacites des sites éoliens',\n        geo_scope = 'europe'\n    )\nfig.show();"

In [167]:
# chargement et traitement des données brutes (II)
rend_offshore_brut = np.genfromtxt('Rendements_offshore.csv', delimiter=',')[:,:N_heures]
rend_onshore_brut = np.genfromtxt('Rendements_onshore.csv', delimiter=',')[:,:N_heures]

print(rend_offshore_brut.shape)
print(rend_onshore_brut.shape)

(642, 292)
(642, 292)


In [168]:
indice_offshore= offshore["index site"]
rend_offshore_matrix = rend_offshore_brut[indice_offshore]
rend_offshore_array = np.array([np.sum(rend_offshore_matrix[i]) for i in range(len(indice_offshore))])


indice_onshore= onshore["index site"]
rend_onshore_matrix = rend_onshore_brut[indice_onshore]
rend_onshore_array = np.array([np.sum(rend_onshore_matrix[i]) for i in range(len(indice_onshore))])

maxoff= max(rend_offshore_array)
maxon = max(rend_onshore_array)

for i in range(len(rend_offshore_array)):
    if (rend_offshore_array[i] == max(rend_offshore_array) ) :
        print("Max offshore = {}  at  index = {} in offshore submatrix".format(maxoff,i))

for i in range(len(rend_onshore_array)):
    if (rend_onshore_array[i] == max(rend_onshore_array) ) : 
        print("Max onshore = {}  at  index = {} in onshore submatrix\n".format(maxon,i))
        
for i in range(642):
    if (rend_offshore_brut[i].sum() == maxoff):
        print("Max offshore = {}  at  index = {} in total matrix".format(maxoff,i))
    elif (rend_onshore_brut[i].sum() == maxon):
        print("Max onshore = {}  at  index = {} in total matrix".format(maxon,i))

Max offshore = 204.69901199999998  at  index = 5 in offshore submatrix
Max onshore = 224.248688  at  index = 233 in onshore submatrix

Max offshore = 204.69901199999998  at  index = 67 in total matrix
Max onshore = 224.248688  at  index = 276 in total matrix


In [169]:
rend_sites_matrix = rend_onshore_brut.copy()   # matrix combinant les valeurs réelles onshore/offshore -> souci de visibilité ci-dessous
for i in np.array(indice_offshore):
    rend_sites_matrix[i] = rend_offshore_brut[i]

print(rend_sites_matrix[443][0] == rend_offshore_brut[443][0])

True


In [170]:
N = np.array([[1,2,3],[5,2,3]])
print(N[:,0])

[1 5]


In [173]:
#version pour le projet

s = CyClpSimplex()

N = N_sites # nombre de sites
H = N_heures # heures
T = 3 # pas de temps
kappa = 0.17
delta = 0.02


# Add variables
x = s.addVariable('x', N)

# Create coefficients and bounds
P=500000

# Add constraints
s += CyLPArray(np.ones(N))*x == P

offshorearray = np.zeros(N)
offshorearray[indice_offshore] = 1      # set à 1 les sites offshore 
s += (CyLPArray(offshorearray)*x) == kappa*P

n_interval = int(N_heures/3) - 1  # nombre d'intervalle sur les heures imparties
print(n_interval)
L = s.addVariable('l', n_interval)  # vecteur T de longeur n_interval pour gérer valeurs absolues 
s += CyLPArray(np.ones(n_interval))*L <= delta*P # contrainte de variabilité
for n in range (n_interval):
    # sum1  = sum2 pour n>0
    if (n == 0) : 
        sum1 = 0
        for i in range (T*n, T*n+T):
            #print(rend_sites_matrix[:,i].shape)
            #print(x.shape)
            #print(np.dot(x, rend_sites_matrix[:,i]).shape)
            sum1 += x*CyLPArray(rend_sites_matrix[:,i])
    #sum1 = 0
    sum2 = 0
    for i in range (T*n, T*n+T):
        #sum1 += CyLPArray(rend_sites_matrix[:,i])*x #- CyLPArray(rend_sites_matrix[:,i+T])*x
        sum2 += CyLPArray(rend_sites_matrix[:,i+T])*x
    s += sum1-sum2 - L[n] <= 0
    s += -sum1+sum2 - L[n] <= 0
    sum1 = sum2

#print ("\nConstraints :\n", s.constraints)
#print ("\nConstraintMatrix :\n", s.coefMatrix)

# Set the objective function
c_off= -CyLPArray(rend_offshore_array)   # rendements associés aux sites offshore
c_on = -CyLPArray(rend_onshore_array)    # rendements associés aux sites onshore
#print(c_off)
#print(c_on)

s.objective = c_off * x[np.array(indice_offshore)] + c_on * x[np.array(indice_onshore)]    #n'accepte pas les types : pandas.series
print("\nc = ",s.objective)

# Solve using primal Simplex
s.primal()

print ("\ntsol = ",s.primalVariableSolution['l']) 
print ("\nxsol = ",s.primalVariableSolution['x']) 
print("\nObjVal = ", -s.objectiveValue)
print("\nSOL OPTIMALE ? = ", s.getStatusString())

96
[-177.0897467  -144.31114151 -158.051434   -156.589402   -128.67606031
 -204.699012   -199.5411832  -192.8983     -189.158542   -164.75720018
 -166.3446258  -154.50272191 -135.09505303 -157.6560369  -163.033075
 -125.53027273  -29.44173075  -41.00356027 -130.2411538  -127.28195223
 -138.08780192 -128.0769466  -119.99245773 -116.3335594  -166.97718829
 -166.77089171 -172.0678092  -145.5604118  -147.6155009  -136.8483987
 -129.7830998  -109.9403389   -90.45257105 -168.7003449  -163.35409949
 -127.5257402  -128.3611811  -136.838765    -60.44128     -47.9047781
 -151.700738    -47.80000676 -192.6386122  -105.05740098 -172.906209
 -121.528569   -158.3158474  -119.87183583 -145.95520025 -135.3507347
 -155.75811    -158.5390776  -183.2684894  -154.220848   -153.3995007
 -118.07917868 -138.4409559  -120.95881695 -135.06498624 -151.8655753
 -152.62977803 -132.3840247  -116.0814573  -126.91619501 -120.65679618
 -116.1411515  -107.73097936 -154.5272845  -199.936666   -167.316809
 -153.0589862 

In [ ]:
"""
var_array = np.zeros(n_interval)
variability = CyLPArray(var_array)
for n in range (n_interval):
    # sum1  = sum2 pour n>0
    if (n == 0) : 
        sum1 = 0
        for k in range (N_sites):
            interv_rend = 0
            for i in range(T*n, T*n+T):
                interv_rend += rend_sites_matrix[k][i]
            sum1 += x[k]*interv_rend
    sum2 = 0
    for k in range (N_sites):
        interv_rend = 0
        for i in range(T*(n+1), T*(n+1)+T):
            interv_rend += rend_sites_matrix[k][i]
        sum1 += x[k]*interv_rend
    
    
    variability[n] = sum1-sum2
    sum1 = sum2
s += variability < delta*P"""

In [75]:
## Visualisation des rendements pour un pays

pays_test = 'France'
index_test = liste_pays_index_dic[pays_test]

# nombre de sites éoliens
index_sites_eoliens = sites[sites.pays==pays_test]['index site']
n_eoliens = len(index_sites_eoliens)
print('il y a '+str(n_eoliens)+' sites éoliens en '+str(pays_test))

# rendements eolien au hasard
random_site = np.random.choice(index_sites_eoliens)
plt.figure(figsize=(9,6))
plt.grid()
plt.title('rendement éolien: '+str(pays_test))
if sites.loc[random_site]['capacite offshore']=='Oui':
    plt.plot(vecteur_temps,matrice_rendements_offshore[random_site])
    plt.legend(['offshore'])
else:
    plt.plot(vecteur_temps,matrice_rendements_onshore[random_site])
    plt.legend(['onshore'])
plt.xlabel('temps : [années]')
plt.ylabel('rendement éolien: [/]')

print('rendement moyen offshore pour le pays: '+str(np.mean(matrice_rendements_offshore[index_sites_eoliens])))
print('rendement moyen onshore pour le pays: '+str(np.mean(matrice_rendements_onshore[index_sites_eoliens])))

# capacité éoliennes maximales tolérables
buf_ok_offshore = (sites.pays==pays_test) & (sites['capacite offshore']=='Oui')
buf_ok_onshore = (sites.pays==pays_test) & (sites['capacite offshore']=='Non')

print(' => capacité éolienne totale offshore : [MW] '+str(sites[buf_ok_offshore]['capacites'].sum()))
print(' => capacité éolienne totale offshore : [MW] '+str(sites[buf_ok_onshore]['capacites'].sum()))

# latitude/longitude
print('le pays '+str(pays_test)+' se trouve  à une latitude de '+str(sites.loc[random_site]['latitude'])+' et une longitude de '+\
      str(sites.loc[random_site]['longitude']))

il y a 80 sites éoliens en France


<IPython.core.display.Javascript object>

NameError: name 'matrice_rendements_onshore' is not defined

In [ ]:
#version template

s = CyClpSimplex()

N = 4 # nombre de sites
H = 10 # heures
T = 2 # pas de temps
offshore = 2  # index à partir duquel on commence à avoir des offshore
kappa = 0.17


# Add variables
x = s.addVariable('x', N)

# Create coefficients and bounds
#p = CyLPArray([100])
p=100


# Add constraints
s += CyLPArray(np.ones(N))*x == p

offshorearray = np.zeros(N)
offshorearray[offshore:] = 1
s += CyLPArray(offshorearray)*x == kappa*p

print ("\nConstraints :\n", s.constraints)
print ("\nConstraintMatrix :\n", s.coefMatrix)

# Set the objective function
#c= -CyLPArray([ i%2 for i in range(N)])
#rend = np.array([[i/(j+40) for j in range(H)] for i in range(N)])
rend = np.array([[1 for j in range(H)] for i in range(N)])
rend_array = np.array([np.sum(rend[i]) for i in range(N)])
c= -CyLPArray(rend_array)
s.objective = c * x
print("\nc = ",s.objective)

# Solve using primal Simplex
s.primal()

print ("\nsol = ",s.primalVariableSolution['x'])
print("\nObjVal = ", -s.objectiveValue)
print("\nSOL OPTIMALE ? = ", s.getStatusString())